In [0]:
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [29]:
from __future__ import absolute_import, division, print_function, unicode_literals

import numpy as np
import pandas as pd
import os 
import json
import codecs

try:
  # %tensorflow_version only exists in Colab.
  %tensorflow_version 2.x
except Exception:
  pass
import tensorflow as tf

from tensorflow import feature_column
from tensorflow import keras

from tensorflow.keras import layers
from sklearn.model_selection import train_test_split

In [30]:
dtypes={
    'Age':                         'int64',
    'Attrition':                   'category',
    'BusinessTravel':              'category',
    'DailyRate':                   'int64',
    'Department':                  'category',
    'DistanceFromHome':            'int64',
    'Education':                   'int64',
    'EducationField':              'category',
    'EmployeeCount':               'int64',
    'EmployeeNumber':              'int64',
    'EnvironmentSatisfaction':     'int64',
    'Gender':                      'category',
    'HourlyRate':                  'int64',
    'JobInvolvement':              'int64',
    'JobLevel':                    'int64',
    'JobRole':                     'category',
    'JobSatisfaction':             'int64',
    'MaritalStatus':               'category',
    'MonthlyIncome':               'int64',
    'MonthlyRate':                 'int64',
    'NumCompaniesWorked':          'int64',
    'Over18':                      'category',
    'OverTime':                    'category',
    'PercentSalaryHike':           'int64',
    'PerformanceRating':           'int64',
    'RelationshipSatisfaction':    'int64',
    'StandardHours':               'int64',
    'StockOptionLevel':            'int64',
    'TotalWorkingYears':           'int64',
    'TrainingTimesLastYear':       'int64',
    'WorkLifeBalance':             'int64',
    'YearsAtCompany':              'int64',
    'YearsInCurrentRole':          'int64',
    'YearsSinceLastPromotion':     'int64',
    'YearsWithCurrManager':        'int64',}

In [42]:

URL = '/content/gdrive/My Drive/WA_Fn-UseC_-HR-Employee-Attrition.xlsx'

df = pd.read_excel('WA_Fn-UseC_-HR-Employee-Attrition.xlsx', sheet_name=0,dtype=dtypes)

In [43]:
# preprocessing : categorical encoding
df['Attrition']=df.Attrition.eq('Yes').mul(1) # change target from Yes/no to 1/0
cont=[]
cat=[]
for key, value in dtypes.items():
    if key!='Attrition':
        if value == "int64":
            cont.append(key)
        else:
            cat.append(key)
df = pd.get_dummies(df, columns=cat)

In [44]:
rename = ["Department_Human Resources",
          "Department_Research & Development" ,
"EducationField_Human Resources" ,
"EducationField_Life Sciences" ,
"EducationField_Technical Degree", 
"JobRole_Healthcare Representative" ,
"JobRole_Human Resources" ,
"JobRole_Laboratory Technician",
"JobRole_Manufacturing Director",
"JobRole_Research Director",
"JobRole_Research Scientist",
"JobRole_Sales Executive",
"JobRole_Sales Representative"]

for string in rename:
  df.rename(columns={ string : string.replace(" ", "_")}, inplace = True)

df.rename(columns={ "Department_Research_&_Development": "Department_ResearchDevelopment"}, inplace = True)
df.rename(columns={ "BusinessTravel_Non-Travel": "BusinessTravel_NonTravel"}, inplace = True)
df.head()








,Age,Attrition,DailyRate,DistanceFromHome,Education,EmployeeCount,EmployeeNumber,EnvironmentSatisfaction,HourlyRate,JobInvolvement,...,JobRole_Research_Director,JobRole_Research_Scientist,JobRole_Sales_Executive,JobRole_Sales_Representative,MaritalStatus_Divorced,MaritalStatus_Married,MaritalStatus_Single,Over18_Y,OverTime_No,OverTime_Yes
0,41,1,1102,1,2,1,1,2,94,3,...,0,0,1,0,0,0,1,1,0,1
1,49,0,279,8,1,1,2,3,61,2,...,0,1,0,0,0,1,0,1,1,0
2,37,1,1373,2,2,1,4,4,92,2,...,0,0,0,0,0,0,1,1,0,1
3,33,0,1392,3,4,1,5,4,56,3,...,0,1,0,0,0,1,0,1,0,1
4,27,0,591,2,1,1,7,1,40,3,...,0,0,0,0,0,1,0,1,1,0


In [45]:
df.dtypes

Age                                  int64
Attrition                            int32
DailyRate                            int64
DistanceFromHome                     int64
Education                            int64
EmployeeCount                        int64
EmployeeNumber                       int64
EnvironmentSatisfaction              int64
HourlyRate                           int64
JobInvolvement                       int64
JobLevel                             int64
JobSatisfaction                      int64
MonthlyIncome                        int64
MonthlyRate                          int64
NumCompaniesWorked                   int64
PercentSalaryHike                    int64
PerformanceRating                    int64
RelationshipSatisfaction             int64
StandardHours                        int64
StockOptionLevel                     int64
TotalWorkingYears                    int64
TrainingTimesLastYear                int64
WorkLifeBalance                      int64
YearsAtComp

In [46]:
cont = []
cat = []

# for items in df.dtypes.iteritems(): 
#   if items[1] == 'int64' and items[0]!='Attrition':
#     cont.append(items[0])
#   else:
#     if items[0] != "Attrition":
#       cat.append(items[0])
# print (cont)
# print (cat)
for items in df.dtypes.iteritems(): 
   if items[0]!='Attrition':
     cont.append(items[0])

print(cont)



['Age', 'DailyRate', 'DistanceFromHome', 'Education', 'EmployeeCount', 'EmployeeNumber', 'EnvironmentSatisfaction', 'HourlyRate', 'JobInvolvement', 'JobLevel', 'JobSatisfaction', 'MonthlyIncome', 'MonthlyRate', 'NumCompaniesWorked', 'PercentSalaryHike', 'PerformanceRating', 'RelationshipSatisfaction', 'StandardHours', 'StockOptionLevel', 'TotalWorkingYears', 'TrainingTimesLastYear', 'WorkLifeBalance', 'YearsAtCompany', 'YearsInCurrentRole', 'YearsSinceLastPromotion', 'YearsWithCurrManager', 'BusinessTravel_NonTravel', 'BusinessTravel_Travel_Frequently', 'BusinessTravel_Travel_Rarely', 'Department_Human_Resources', 'Department_ResearchDevelopment', 'Department_Sales', 'EducationField_Human_Resources', 'EducationField_Life_Sciences', 'EducationField_Marketing', 'EducationField_Medical', 'EducationField_Other', 'EducationField_Technical_Degree', 'Gender_Female', 'Gender_Male', 'JobRole_Healthcare_Representative', 'JobRole_Human_Resources', 'JobRole_Laboratory_Technician', 'JobRole_Manager

In [47]:
train, test = train_test_split(df, test_size=0.2)
train, val = train_test_split(train, test_size=0.2)
print(len(train), 'train examples')
print(len(val), 'validation examples')
print(len(test), 'test examples')

940 train examples
236 validation examples
294 test examples


In [48]:

# A utility method to create a tf.data dataset from a Pandas Dataframe
def df_to_dataset(dataframe, shuffle=True, batch_size=32):
  dataframe = dataframe.copy()
  labels = dataframe.pop('Attrition')
  ds = tf.data.Dataset.from_tensor_slices((dict(dataframe), labels))
  if shuffle:
    ds = ds.shuffle(buffer_size=len(dataframe))
  ds = ds.batch(batch_size)
  return ds

In [49]:
batch_size = 5 # A small batch sized is used for demonstration purposes
train_ds = df_to_dataset(train, batch_size=batch_size)
val_ds = df_to_dataset(val, shuffle=False, batch_size=batch_size)
test_ds = df_to_dataset(test, shuffle=False, batch_size=batch_size)

In [50]:
for feature_batch, label_batch in train_ds.take(1):
  print('Every feature:', list(feature_batch.keys()))
  print('A batch of Education:', feature_batch['Education'])
  print('A batch of Attrication:', label_batch )

Every feature: ['Age', 'DailyRate', 'DistanceFromHome', 'Education', 'EmployeeCount', 'EmployeeNumber', 'EnvironmentSatisfaction', 'HourlyRate', 'JobInvolvement', 'JobLevel', 'JobSatisfaction', 'MonthlyIncome', 'MonthlyRate', 'NumCompaniesWorked', 'PercentSalaryHike', 'PerformanceRating', 'RelationshipSatisfaction', 'StandardHours', 'StockOptionLevel', 'TotalWorkingYears', 'TrainingTimesLastYear', 'WorkLifeBalance', 'YearsAtCompany', 'YearsInCurrentRole', 'YearsSinceLastPromotion', 'YearsWithCurrManager', 'BusinessTravel_NonTravel', 'BusinessTravel_Travel_Frequently', 'BusinessTravel_Travel_Rarely', 'Department_Human_Resources', 'Department_ResearchDevelopment', 'Department_Sales', 'EducationField_Human_Resources', 'EducationField_Life_Sciences', 'EducationField_Marketing', 'EducationField_Medical', 'EducationField_Other', 'EducationField_Technical_Degree', 'Gender_Female', 'Gender_Male', 'JobRole_Healthcare_Representative', 'JobRole_Human_Resources', 'JobRole_Laboratory_Technician', '

In [51]:
# We will use this batch to demonstrate several types of feature columns
example_batch = next(iter(train_ds))[0]

In [52]:

# A utility method to create a feature column
# and to transform a batch of data
def demo(feature_column):
  feature_layer = layers.DenseFeatures(feature_column)
  print(feature_layer(example_batch).numpy())

In [53]:
daysOpen = feature_column.numeric_column("Education")
demo(daysOpen)

[[2.]
 [3.]
 [1.]
 [4.]
 [1.]]


In [54]:
feature_columns = []


dep_var = 'Attrition'
# cat_names = ['RequestorSeniority', 'FiledAgainst', 'TicketType', 'Severity', 'Priority']
# emb_names = ['Requestor', 'ITOwner']
# cont_names = ['daysOpen']

# numeric cols
for header in cont:
  if header != 'Attrition':
    feature_columns.append(feature_column.numeric_column(header))


# indicator cols
# for header in cat:
#   if header != 'Attrition':
#     feature = feature_column.categorical_column_with_vocabulary_list(header, df[header].unique())
#     feature_one_hot = feature_column.indicator_column(feature)
#     feature_columns.append(feature_one_hot)

  
# # Calc Dimension for embbed cloumn
# for header in emb_names:
#   feature = feature_column.categorical_column_with_vocabulary_list(header, dataframe[header].unique())
#   # embedding_dimensions =  number_of_categories**0.25
#   embedding_dimensions = dataframe[header].nunique()**0.25
#   feature_embedding = feature_column.embedding_column(feature, dimension=int(embedding_dimensions))

 

In [55]:
print(feature_columns)

[NumericColumn(key='Age', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None), NumericColumn(key='DailyRate', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None), NumericColumn(key='DistanceFromHome', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None), NumericColumn(key='Education', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None), NumericColumn(key='EmployeeCount', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None), NumericColumn(key='EmployeeNumber', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None), NumericColumn(key='EnvironmentSatisfaction', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None), NumericColumn(key='HourlyRate', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None), NumericColumn(key='JobInvolvement', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None), NumericColumn(key='JobLevel', shape=(1,), default_v

In [56]:
feature_layer = tf.keras.layers.DenseFeatures(feature_columns)

In [57]:
batch_size = 32
train_ds = df_to_dataset(train, batch_size=batch_size)
val_ds = df_to_dataset(val, shuffle=False, batch_size=batch_size)
test_ds = df_to_dataset(test, shuffle=False, batch_size=batch_size)

In [58]:
for feature_batch, label_batch in train_ds.take(1):
  print('Every feature:', list(feature_batch.keys()))
  print('A batch of Education:', feature_batch['Education'])
  print('A batch of Attration:', label_batch )

Every feature: ['Age', 'DailyRate', 'DistanceFromHome', 'Education', 'EmployeeCount', 'EmployeeNumber', 'EnvironmentSatisfaction', 'HourlyRate', 'JobInvolvement', 'JobLevel', 'JobSatisfaction', 'MonthlyIncome', 'MonthlyRate', 'NumCompaniesWorked', 'PercentSalaryHike', 'PerformanceRating', 'RelationshipSatisfaction', 'StandardHours', 'StockOptionLevel', 'TotalWorkingYears', 'TrainingTimesLastYear', 'WorkLifeBalance', 'YearsAtCompany', 'YearsInCurrentRole', 'YearsSinceLastPromotion', 'YearsWithCurrManager', 'BusinessTravel_NonTravel', 'BusinessTravel_Travel_Frequently', 'BusinessTravel_Travel_Rarely', 'Department_Human_Resources', 'Department_ResearchDevelopment', 'Department_Sales', 'EducationField_Human_Resources', 'EducationField_Life_Sciences', 'EducationField_Marketing', 'EducationField_Medical', 'EducationField_Other', 'EducationField_Technical_Degree', 'Gender_Female', 'Gender_Male', 'JobRole_Healthcare_Representative', 'JobRole_Human_Resources', 'JobRole_Laboratory_Technician', '

In [59]:
model = tf.keras.Sequential([
  feature_layer,
  layers.Dense(64, activation='relu'),
  layers.Dense(64, activation='relu'),
  layers.Dense(1, activation='sigmoid')
])

model.compile(optimizer=keras.optimizers.Adam(lr=1e-3),
              loss=keras.losses.BinaryCrossentropy(),
              metrics=['accuracy'],
              run_eagerly=True)

model.fit(train_ds,
          validation_data=val_ds,
          epochs=5)

Epoch 1/5
30/30 [==============================] - 2s 53ms/step - loss: 2.7208 - accuracy: 0.8234 - val_loss: 0.0000e+00 - val_accuracy: 0.0000e+00
Epoch 2/5
30/30 [==============================] - 1s 48ms/step - loss: 2.7259 - accuracy: 0.8234 - val_loss: 1.9884 - val_accuracy: 0.8602
Epoch 3/5
30/30 [==============================] - 1s 46ms/step - loss: 2.7098 - accuracy: 0.8234 - val_loss: 1.9884 - val_accuracy: 0.8602
Epoch 4/5
30/30 [==============================] - 1s 48ms/step - loss: 2.7098 - accuracy: 0.8234 - val_loss: 1.9884 - val_accuracy: 0.8602
Epoch 5/5
30/30 [==============================] - 1s 48ms/step - loss: 2.7098 - accuracy: 0.8234 - val_loss: 1.9884 - val_accuracy: 0.8602


In [60]:

loss, accuracy = model.evaluate(test_ds)
print("Accuracy", accuracy)

10/10 [==============================] - 0s 37ms/step - loss: 1.8317 - accuracy: 0.8707
Accuracy 0.8707483


In [62]:
i = 0
df2 =  df.drop(['Attrition'], axis=1)
x = np.array(df2.iloc[i])
x = x.reshape(-1,1)
y = x.tolist()
print(y)
file_path = "/content/gdrive/My Drive/hrtest.json"
#json.dump(y, codecs.open(file_path, 'w', encoding='utf-8'), separators=(',', ':'), sort_keys=True, indent=4)

[[41], [1102], [1], [2], [1], [1], [2], [94], [3], [2], [4], [5993], [19479], [8], [11], [3], [1], [80], [0], [8], [0], [1], [6], [4], [0], [5], [0], [0], [1], [0], [0], [1], [0], [1], [0], [0], [0], [0], [1], [0], [0], [0], [0], [0], [0], [0], [0], [1], [0], [0], [0], [1], [1], [0], [1]]


In [63]:
model.predict(y)

ValueError: Error when checking model input: the list of Numpy arrays that you are passing to your model is not the size the model expected. Expected to see 55 array(s), but instead got the following list of 1 arrays: [<tf.Tensor: id=328923, shape=(32, 1), dtype=int32, numpy=
array([[   41],
       [ 1102],
       [    1],
       [    2],
       [    1],
       [    1],
       [    2],
       [   94],
       [    3...

In [0]:
model.summary()
model.save('/content/gdrive/My Drive/tabular/saved_model') 

Model: "sequential_14"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_features_106 (DenseFea multiple                  0         
_________________________________________________________________
dense_42 (Dense)             multiple                  4736      
_________________________________________________________________
dense_43 (Dense)             multiple                  16512     
_________________________________________________________________
dense_44 (Dense)             multiple                  129       
Total params: 21,377
Trainable params: 21,377
Non-trainable params: 0
_________________________________________________________________
INFO:tensorflow:Assets written to: /content/gdrive/My Drive/tabular/saved_model/assets


In [0]:
!saved_model_cli show --dir '/content/gdrive/My Drive/tabular/saved_model/1' --all


MetaGraphDef with tag-set: 'serve' contains the following SignatureDefs:

signature_def['__saved_model_init_op']:
  The given SavedModel SignatureDef contains the following input(s):
  The given SavedModel SignatureDef contains the following output(s):
    outputs['__saved_model_init_op'] tensor_info:
        dtype: DT_INVALID
        shape: unknown_rank
        name: NoOp
  Method name is: 

signature_def['serving_default']:
  The given SavedModel SignatureDef contains the following input(s):
    inputs['Age'] tensor_info:
        dtype: DT_INT32
        shape: (-1, 1)
        name: serving_default_Age:0
    inputs['BusinessTravel_NonTravel'] tensor_info:
        dtype: DT_INT32
        shape: (-1, 1)
        name: serving_default_BusinessTravel_NonTravel:0
    inputs['BusinessTravel_Travel_Frequently'] tensor_info:
        dtype: DT_INT32
        shape: (-1, 1)
        name: serving_default_BusinessTravel_Travel_Frequently:0
    inputs['BusinessTravel_Travel_Rarely'] tensor_info:
  

In [0]:
!echo "deb http://storage.googleapis.com/tensorflow-serving-apt stable tensorflow-model-server tensorflow-model-server-universal" | tee /etc/apt/sources.list.d/tensorflow-serving.list && \
curl https://storage.googleapis.com/tensorflow-serving-apt/tensorflow-serving.release.pub.gpg | apt-key add -
!apt update

In [0]:
!apt-get install tensorflow-model-server


In [0]:
os.environ["MODEL_DIR"] = '/content/gdrive/My Drive/tabular/saved_model/'


In [0]:
%%bash --bg 
nohup tensorflow_model_server \
  --rest_api_port=8501 \
  --model_name=saved_model \
  --model_base_path="${MODEL_DIR}" >server.log 2>&1

Starting job # 5 in a separate thread.


In [0]:
!tail server.log


IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



In [0]:
URL = '/content/gdrive/My Drive/WA_Fn-UseC_-HR-Employee-Attrition.xlsx'
dataframe = pd.read_excel('/content/gdrive/My Drive/WA_Fn-UseC_-HR-Employee-Attrition.xlsx', sheet_name=0,dtype=dtypes)
dataframe['Attrition']=dataframe.Attrition.eq('Yes').mul(1) # change target from Yes/no to 1/0
dataframe.head()

,Age,Attrition,BusinessTravel,DailyRate,Department,DistanceFromHome,Education,EducationField,EmployeeCount,EmployeeNumber,EnvironmentSatisfaction,Gender,HourlyRate,JobInvolvement,JobLevel,JobRole,JobSatisfaction,MaritalStatus,MonthlyIncome,MonthlyRate,NumCompaniesWorked,Over18,OverTime,PercentSalaryHike,PerformanceRating,RelationshipSatisfaction,StandardHours,StockOptionLevel,TotalWorkingYears,TrainingTimesLastYear,WorkLifeBalance,YearsAtCompany,YearsInCurrentRole,YearsSinceLastPromotion,YearsWithCurrManager
0,41,1,Travel_Rarely,1102,Sales,1,2,Life Sciences,1,1,2,Female,94,3,2,Sales Executive,4,Single,5993,19479,8,Y,Yes,11,3,1,80,0,8,0,1,6,4,0,5
1,49,0,Travel_Frequently,279,Research & Development,8,1,Life Sciences,1,2,3,Male,61,2,2,Research Scientist,2,Married,5130,24907,1,Y,No,23,4,4,80,1,10,3,3,10,7,1,7
2,37,1,Travel_Rarely,1373,Research & Development,2,2,Other,1,4,4,Male,92,2,1,Laboratory Technician,3,Single,2090,2396,6,Y,Yes,15,3,2,80,0,7,3,3,0,0,0,0
3,33,0,Travel_Frequently,1392,Research & Development,3,4,Life Sciences,1,5,4,Female,56,3,1,Research Scientist,3,Married,2909,23159,1,Y,Yes,11,3,3,80,0,8,3,3,8,7,3,0
4,27,0,Travel_Rarely,591,Research & Development,2,1,Medical,1,7,1,Male,40,3,1,Laboratory Technician,2,Married,3468,16632,9,Y,No,12,3,4,80,1,6,3,3,2,2,2,2


In [0]:
cont=[]
cat=[]
for key, value in dtypes.items():
    if key!='Attrition':
        if value == "int64":
            cont.append(key)
        else:
            cat.append(key)

In [0]:
print(cat)

['BusinessTravel', 'Department', 'EducationField', 'Gender', 'JobRole', 'MaritalStatus', 'Over18', 'OverTime']


In [0]:
dataframe['BusinessTravel'].unique()

[Travel_Rarely, Travel_Frequently, Non-Travel]
Categories (3, object): [Travel_Rarely, Travel_Frequently, Non-Travel]

In [0]:
train, test = train_test_split(dataframe, test_size=0.2)
train, val = train_test_split(train, test_size=0.2)
print(len(train), 'train examples')
print(len(val), 'validation examples')
print(len(test), 'test examples')

940 train examples
236 validation examples
294 test examples


In [0]:
# A utility method to create a tf.data dataset from a Pandas Dataframe
def df_to_dataset(dataframe, shuffle=True, batch_size=32):
  dataframe = dataframe.copy()
  labels = dataframe.pop('Attrition')
  ds = tf.data.Dataset.from_tensor_slices((dict(dataframe), labels))
  if shuffle:
    ds = ds.shuffle(buffer_size=len(dataframe))
  ds = ds.batch(batch_size)
  return ds

In [0]:
example_batch = next(iter(train_ds))[0]


In [0]:
# A utility method to create a feature column
# and to transform a batch of data
def demo(feature_column):
  feature_layer = layers.DenseFeatures(feature_column)
  print(feature_layer(example_batch).numpy())